In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

Using TensorFlow backend.


In [2]:
data = DataLoader(config_mixed_flickr8k_flickr8k_polish)

Loading dataset
Loading train dataset
Loading images splits
Images splits loaded
Number of train images:  6000
Number of test images:  1000
Loading all captions
All captions loaded
Nuber of all captions:  8091
Loading captions splits
Captions splits loaded
Number of train captions:  6000
Number of test test:  1000
Loading test dataset
Loading images splits
Images splits loaded
Number of train images:  6000
Number of test images:  1000
Loading all captions
All captions loaded
Nuber of all captions:  8091
Loading captions splits
Captions splits loaded
Number of train captions:  6000
Number of test test:  1000


In [3]:
data=preprocess_data(data)

Number of train images:  6000
Number of test images:  1000
Number of train captions:  6000
Number of test captions:  1000


/Users/mateuszb/opt/anaconda3/envs/framework/lib/python3.7/site-packages/spacy/util.py:845: UserWarning: [W094] Model 'pl_model' (0.1.0) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.3.1,<3.4.0
  warnings.warn(warn_msg)


OSError: [E053] Could not read config file from /Users/mateuszb/opt/anaconda3/envs/framework/lib/python3.7/site-packages/pl_spacy_model/pl_model-0.1.0/config.cfg

In [ ]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, \
    Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.models import Model
from keras import Input, layers
from keras import callbacks
from eval_utils import calculate_results, prepare_for_evaluation
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array
import pickle
from eval_utils import generate_report

In [ ]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    """
    Data generator, that serves the data to the model during training


    Parameters
    ----------
    descriptions: str
        Dictionary where key is image id and value is a list of preprocessed descriptions
    photos
        Dictionary with encoded images(vector of image features extracted by specified image feature extractor
         fe. Inception), identified by image id
    wordtoix
        Dictionary with keys-words , values -id of word
    max_length
        Max number of words in caption on dataset
    num_photos_per_batch: int
    vocab_size: int
    Returns
    -------
    """
    X1, X2, y = list(), list(), list()
    n = 0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n += 1
            # retrieve the photo feature from the dictionary
            photo = photos[key]
            for desc in desc_list:
                # encode the sentence by translating it to the number representation,
                # with the dictionary of words created in in the previous stage
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n = 0

class ModelImpl:
    def __init__(self, data):
        self.data=data
        inputs1 = Input(shape=(2048,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation='relu')(fe1)
        inputs2 = Input(shape=(self.data.max_length,))
        se1 = Embedding(self.data.vocab_size, general[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation='relu')(decoder1)
        outputs = Dense(self.data.vocab_size, activation='softmax')(decoder2)
        self.model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        self.model.summary()
        self.model.layers[2]

        self.model.layers[2].set_weights([self.data.embedding_matrix])
        self.model.layers[2].trainable = False

        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer())
        self.setup()

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 2
        self.number_pics_per_bath = 100
        self.steps = len(self.data.train_captions_wrapped) // self.number_pics_per_bath

    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            callback = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            generator = data_generator(self.data.train_captions_wrapped,
                                       self.data.encoded_images_train,
                                       self.data.wordtoix,
                                       self.data.max_length,
                                       self.number_pics_per_bath,
                                       self.data.vocab_size)
            self.model.fit(generator, epochs=self.epochs,
                           steps_per_epoch=self.steps,
                           callbacks=[callback],
                           verbose=1)
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoded_images_test, self.data.test_captions_mapping,
                                                   self.data.wordtoix, self.data.ixtoword, self.data.max_length,
                                                   self.model)
        out = calculate_results(expected, results, self.data.configuration)
        print(out)

model=ModelImpl(data)


In [ ]:
model.train()

In [ ]:
model.evaluate()

In [ ]:
generate_report(general["results_directory"])
